In [10]:
###NOT FINISHED####
###problems with reddit api - it is paid now and requires extra paperwork
import openai
from dotenv import load_dotenv
import sys
import os
import pandas as pd
import praw
from typing import List, Dict, Generator

In [4]:
load_dotenv(".env")
openai.api_key = os.environ["API"]
reddit = praw.Reddit(
    client_id=os.environ["REDDIT_CLIENT"],
    client_secret=os.environ["REDDIT_SECRET"],
    user_agent=f"script:test:0.0.1 (by u/purl_knittingpat)",
)

In [5]:
DF_COLUMNS = ["subreddit", "submission_id", "score", "comment_body"]

filename, subreddits = "cities.csv", [
    "NYC",
    "Seattle",
    "LosAngeles",
    "Chicago",
    "Austin",
    "Portland",
    "SanFrancisco",
    "Boston",
    "Houston",
    "Atlanta",
    "Philadelphia",
    "Denver",
    "SeattleWa",
    "Dallas",
    "WashingtonDC",
    "SanDiego",
    "Pittsburgh",
    "Phoenix",
    "Minneapolis",
    "Orlando",
    "Nashville",
    "StLouis",
    "SaltLakeCity",
    "Columbus",
    "Raleigh",
]

In [11]:
# Utility functions for fetching comments from submissions
def comment_generator(submission) -> Generator:
    # Do not bother expanding MoreComments (follow-links)
    for comment in submission.comments.list():
        if hasattr(comment, "body") and comment.body != "[deleted]" and comment.body != "[removed]":
            yield (comment)

In [12]:
def collect_comments(
    filename: str,
    target_comments_per_subreddit: int,
    max_comments_per_submission: int,
    max_comment_length: int,
    reddit: praw.Reddit,
) -> pd.DataFrame:
    """
    Collect comments from the top submissions in each subreddit.

    Cache results at cache_filename.

    Return a dataframe with columns: subreddit, submission_id, score, comment_body
    """
    try:
        df = pd.read_csv(filename, index_col="id")
        assert df.columns.tolist() == DF_COLUMNS
    except FileNotFoundError:
        df = pd.DataFrame(columns=DF_COLUMNS)

    # dict like {comment_id -> {column -> value}}
    records = df.to_dict(orient="index")

    for subreddit_index, subreddit_name in enumerate(subreddits):
        print(f"Processing Subreddit: {subreddit_name}")

        processed_comments_for_subreddit = len(df[df["subreddit"] == subreddit_name])

        if processed_comments_for_subreddit >= target_comments_per_subreddit:
            print(f"Enough comments fetched for {subreddit_name}, continuing to next subreddit.")
            continue

        # `top`` is a generator, grab submissions until we break (within this loop).
        for submission in reddit.subreddit(subreddit_name).top(time_filter="month"):
            if processed_comments_for_subreddit >= target_comments_per_subreddit:
                break

            # The number of comments that we already have for this subreddit
            processed_comments_for_submission = len(df[df["submission_id"] == submission.id])

            for comment in comment_generator(submission):
                if processed_comments_for_submission >= max_comments_per_submission or processed_comments_for_subreddit >= target_comments_per_subreddit:
                    break

                if comment.id in records:
                    print(f"Skipping comment {subreddit_name}-{submission.id}-{comment.id} because we already have it")
                    continue

                body = comment.body[:max_comment_length].strip()
                records[comment.id] = {"subreddit": subreddit_name, "submission_id": submission.id, "comment_body": body}

                processed_comments_for_subreddit += 1
                processed_comments_for_submission += 1

            # Once per post write to disk.
            print(f"CSV rewritten with {len(records)} rows.\n")
            df = pd.DataFrame.from_dict(records, orient="index", columns=DF_COLUMNS)
            df.to_csv(filename, index_label="id")

    print("Completed.")
    return df

In [13]:
NUM_SUBREDDITS = len(subreddits)
TARGET_COMMENTS_PER_SUBREDDIT = 50
MAX_COMMENTS_PER_SUBMISSION = 10
MAX_COMMENT_LENGTH = 2000

collect_comments(
    filename=filename,
    target_comments_per_subreddit=TARGET_COMMENTS_PER_SUBREDDIT,
    max_comments_per_submission=MAX_COMMENTS_PER_SUBMISSION,
    max_comment_length=MAX_COMMENT_LENGTH,
    reddit=reddit,
)

Processing Subreddit: NYC
CSV rewritten with 10 rows.

CSV rewritten with 20 rows.

CSV rewritten with 30 rows.

CSV rewritten with 40 rows.

CSV rewritten with 50 rows.

Processing Subreddit: Seattle
CSV rewritten with 60 rows.

CSV rewritten with 70 rows.

CSV rewritten with 80 rows.

CSV rewritten with 90 rows.

CSV rewritten with 100 rows.

Processing Subreddit: LosAngeles
CSV rewritten with 110 rows.

CSV rewritten with 120 rows.

CSV rewritten with 130 rows.

CSV rewritten with 140 rows.

CSV rewritten with 150 rows.

Processing Subreddit: Chicago
CSV rewritten with 160 rows.

CSV rewritten with 170 rows.

CSV rewritten with 180 rows.

CSV rewritten with 190 rows.

CSV rewritten with 200 rows.

Processing Subreddit: Austin
CSV rewritten with 210 rows.

CSV rewritten with 220 rows.

CSV rewritten with 230 rows.

CSV rewritten with 240 rows.

CSV rewritten with 250 rows.

Processing Subreddit: Portland
CSV rewritten with 260 rows.

CSV rewritten with 270 rows.

CSV rewritten with 2

,subreddit,submission_id,score,comment_body
k2q1hf9,NYC,16vc42t,NaN,I certainly feel empathy for those with baseme...
k2qbken,NYC,16vc42t,NaN,"RIP people moving this weekend, since it's the..."
k2q9tir,NYC,16vc42t,NaN,Damn. That’s someone’s $3000 studio basement ...
k2q7a36,NYC,16vc42t,NaN,Empire Blvd and Flatbush was flooded and that ...
k2q9qyc,NYC,16vc42t,NaN,The subways are definitely flooding today.
...,...,...,...,...
k46yme1,Raleigh,1740lia,NaN,Went there for a Disney on Ice and searched fo...
k46hq3e,Raleigh,1740lia,NaN,"Don't forget, taxpayers paid for that building..."
k46lbcq,Raleigh,1740lia,NaN,You can walk up to the 3rd floor without needi...
k47lq5r,Raleigh,1740lia,NaN,6.00 water is a crime


In [14]:
pd.read_csv(filename)

,id,subreddit,submission_id,score,comment_body
0,k2q1hf9,NYC,16vc42t,NaN,I certainly feel empathy for those with baseme...
1,k2qbken,NYC,16vc42t,NaN,"RIP people moving this weekend, since it's the..."
2,k2q9tir,NYC,16vc42t,NaN,Damn. That’s someone’s $3000 studio basement ...
3,k2q7a36,NYC,16vc42t,NaN,Empire Blvd and Flatbush was flooded and that ...
4,k2q9qyc,NYC,16vc42t,NaN,The subways are definitely flooding today.
...,...,...,...,...,...
1245,k46yme1,Raleigh,1740lia,NaN,Went there for a Disney on Ice and searched fo...
1246,k46hq3e,Raleigh,1740lia,NaN,"Don't forget, taxpayers paid for that building..."
1247,k46lbcq,Raleigh,1740lia,NaN,You can walk up to the 3rd floor without needi...
1248,k47lq5r,Raleigh,1740lia,NaN,6.00 water is a crime
